In [ ]:
!pip install transformers
!pip install datasets

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, GPT2Config
from transformers import Trainer, TrainingArguments
from transformers import AdamW, get_cosine_schedule_with_warmup
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
dataset = load_dataset('carblacac/twitter-sentiment-analysis') # https://huggingface.co/datasets/carblacac/twitter-sentiment-analysis

train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-620063a91eb1d8cd/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


...

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29997 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/61998 [00:00<?, ? examples/s]

Dataset twitter-sentiment-analysis downloaded and prepared to /root/.cache/huggingface/datasets/carblacac___twitter-sentiment-analysis/default/1.0.0/cd65e23e456de6a4f7264e305380b0ffe804d6f5bfd361c0ec0f68d8d1fab95b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train_texts = train_data['text']
train_labels = train_data['feeling']

val_texts = val_data['text']
val_labels = val_data['feeling']

test_texts = test_data['text']
test_labels = test_data['feeling']

In [5]:
len(train_texts), len(val_texts), len(test_texts)

(119988, 29997, 61998)

In [6]:
train_texts = train_texts[:10000]
train_labels = train_labels[:10000]
val_texts = val_texts[:1000]
val_labels = val_labels[:1000]
test_texts = test_texts[:2000]
test_labels = test_labels[:2000]

In [7]:
len(train_texts), len(val_texts), len(test_texts)

(10000, 1000, 2000)

# GPT2

In [8]:
model_config = GPT2Config.from_pretrained('gpt2', num_labels=2)
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=model_config)

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels):
        super().__init__()
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        return {'text': text, 'label': label}

train_dataset = TextClassificationDataset(train_texts, train_labels)
val_dataset = TextClassificationDataset(val_texts, val_labels)
test_dataset = TextClassificationDataset(test_texts, test_labels)

In [10]:
class DataCollator(object):
    def __init__(self, tokenizer, max_seq_len=None):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        return
    
    def __call__(self, sequences):
        texts = [sequence['text'] for sequence in sequences]
        labels = [int(sequence['label']) for sequence in sequences]
        inputs = self.tokenizer(text=texts,
                                return_tensors='pt',
                                padding=True,
                                truncation=True,
                                max_length=self.max_seq_len)
        inputs.update({'labels': torch.tensor(labels)})
        return inputs

datacollator = DataCollator(tokenizer=tokenizer, max_seq_len=70)

In [11]:
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=32,
                              shuffle=True,
                              collate_fn=datacollator)
val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=32,
                            shuffle=False,
                            collate_fn=datacollator)
test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=32,
                             shuffle=False,
                             collate_fn=datacollator)

In [12]:
total_epochs = 5

optimizer = AdamW(model.parameters(), lr=1e-5)

num_train_steps = len(train_dataloader) * total_epochs
num_warmup_steps = int(num_train_steps * 0.1) 

lr_scheduler = get_cosine_schedule_with_warmup(optimizer,
                                               num_warmup_steps=num_warmup_steps,
                                               num_training_steps = num_train_steps)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
def train(dataloader, optimizer, scheduler, device):
    global model
    model.train()
    model.to(device)
    
    prediction_labels = []
    true_labels = []
    
    total_loss = []
    
    for batch in dataloader:
        true_labels += batch['labels'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss, logits = outputs[:2]
        logits = logits.detach().cpu().numpy()
        total_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # prevent exploding gradient

        optimizer.step()
        scheduler.step()
        
        prediction_labels += logits.argmax(axis=-1).flatten().tolist()
    
    return true_labels, prediction_labels, total_loss

def validation(dataloader, device):
    global model
    model.eval()
    model.to(device)
    
    prediction_labels = []
    true_labels = []
    
    total_loss = []
    
    for batch in dataloader:
        true_labels += batch['labels'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device) for k, v in batch.items()}
        
        with torch.no_grad():
            outputs = model(**batch)
            loss, logits = outputs[:2]
            logits = logits.detach().cpu().numpy()
            total_loss.append(loss.item())

            prediction_labels += logits.argmax(axis=-1).flatten().tolist()
        
    return true_labels, prediction_labels, total_loss

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

all_loss = {'train_loss': [], 'val_loss': []}
all_acc = {'train_acc': [], 'val_acc': []}

for epoch in range(total_epochs):
    y, y_pred, train_loss = train(train_dataloader, optimizer, lr_scheduler, device)
    train_acc = accuracy_score(y, y_pred)
    
    y, y_pred, val_loss = validation(val_dataloader, device)
    val_acc = accuracy_score(y, y_pred)
    
    all_loss['train_loss'] += train_loss
    all_loss['val_loss'] += val_loss
    
    all_acc['train_acc'].append(train_acc)
    all_acc['val_acc'].append(val_acc)
    
    print(f'Epoch: {epoch}, train_loss: {torch.tensor(train_loss).mean():.3f}, train_acc: {train_acc:.3f}, val_loss: {torch.tensor(val_loss).mean():.3f}, val_acc: {val_acc:.3f}') 

Epoch: 0, train_loss: 1.628, train_acc: 0.504, val_loss: 0.868, val_acc: 0.518
Epoch: 1, train_loss: 1.190, train_acc: 0.515, val_loss: 0.733, val_acc: 0.526
Epoch: 2, train_loss: 0.981, train_acc: 0.515, val_loss: 0.709, val_acc: 0.550
Epoch: 3, train_loss: 0.964, train_acc: 0.524, val_loss: 0.713, val_acc: 0.534
Epoch: 4, train_loss: 0.929, train_acc: 0.521, val_loss: 0.711, val_acc: 0.538


In [15]:
_, y_pred, _ = validation(test_dataloader, device)
print("Accuracy: {}".format(accuracy_score(test_labels, y_pred)))

Accuracy: 0.529


In [16]:
model.eval()
input_text = "I really love machine learning"
inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
print(f"Predicted class: {predicted_class}")

Predicted class: 1
